In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
np.set_printoptions(precision=2)
from sklearn.metrics import r2_score  # Make sure this import is included


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score

# Load training and testing data
train_df = pd.read_csv(r'C:\Users\User\Work\College\AIMS\KAGGLE COMP 2\aims-dtu-discord-machine-learning-challenge\train.csv')
test_df = pd.read_csv(r'C:\Users\User\Work\College\AIMS\KAGGLE COMP 2\aims-dtu-discord-machine-learning-challenge\test.csv')

# Separate features and target in the training data
X_train = train_df.drop(['flood_probability'], axis=1)
y_train = train_df['flood_probability']

# Extract features from the test data (without 'flood_probability' since it's not present in the test set)
X_test = test_df.drop(['flood_probability'], axis=1, errors='ignore')

# Normalize the data using StandardScaler
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

# Train SGDRegressor
sgdr = SGDRegressor(max_iter=1000)
sgdr.fit(X_train_norm, y_train)
print(sgdr)
print(f"Number of iterations completed: {sgdr.n_iter_}, Number of weight updates: {sgdr.t_}")

# Model parameters
b_norm = sgdr.intercept_
w_norm = sgdr.coef_
print(f"Model parameters: w: {w_norm}, b: {b_norm}")

# Make predictions on the test set
y_test_pred = sgdr.predict(X_test_norm)

# Print R² score (can be skipped since R² score is for training set comparison)
# R² score on training set
y_train_pred = sgdr.predict(X_train_norm)
r2 = r2_score(y_train, y_train_pred)
print(f"R² score: {r2}")

# Create the submission DataFrame
submission = pd.DataFrame({'id': test_df['id'], 'flood_probability': y_test_pred})

# Check if the submission has the correct number of rows (745,305)
required_rows = 745305
current_rows = len(submission)

if current_rows < required_rows:
    # Replicate rows to match the required submission size
    rows_to_add = required_rows - current_rows
    submission = pd.concat([submission] * (rows_to_add // current_rows), ignore_index=True)
    submission = submission.head(required_rows)  # Ensure exact number of rows

# Save the generated submission as a CSV
submission.to_csv(r'C:\Users\User\Work\College\AIMS\KAGGLE COMP 2\generated_submission.csv', index=False)

print("Sample submission generated and saved as 'generated_submission.csv'.")


SGDRegressor()
Number of iterations completed: 6, Number of weight updates: 6707743.0
Model parameters: w: [-0.    0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01
  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01], b: [0.5]
R² score: 0.844623088295565
Sample submission generated and saved as 'generated_submission.csv'.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

# Load data
train_df = pd.read_csv('aims-dtu-discord-machine-learning-challenge/train.csv')
test_df = pd.read_csv('aims-dtu-discord-machine-learning-challenge/test.csv')

# Print column names to understand the data structure
print("Training data columns:", train_df.columns.tolist())

def create_features(df):
    # Create a copy to avoid modifying the original dataframe
    df = df.copy()
    
    # Get numerical columns (excluding 'id' and target variable)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'id' in num_cols:
        num_cols.remove('id')
    if 'flood_probability' in num_cols:
        num_cols.remove('flood_probability')
    
    # Create interactions between numerical features
    for i in range(len(num_cols)):
        for j in range(i+1, len(num_cols)):
            col1, col2 = num_cols[i], num_cols[j]
            df[f'{col1}_{col2}_interact'] = df[col1] * df[col2]
            df[f'{col1}_{col2}_ratio'] = df[col1] / (df[col2] + 1e-8)
    
    # Create polynomial features for each numerical column
    for col in num_cols:
        df[f'{col}_squared'] = df[col] ** 2
        df[f'{col}_cubed'] = df[col] ** 3
    
    return df

# Feature engineering
X_train = create_features(train_df)
X_test = create_features(test_df)

# Prepare target
y_train = X_train['flood_probability']
X_train = X_train.drop(['flood_probability', 'id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create ensemble of models
models = {
    'xgb': xgb.XGBRegressor(
        n_estimators=2000,
        learning_rate=0.005,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),
    'gbm': GradientBoostingRegressor(
        n_estimators=1000,
        learning_rate=0.005,
        max_depth=7,
        random_state=42
    ),
    'rf': RandomForestRegressor(
        n_estimators=1000,
        max_depth=20,
        min_samples_split=5,
        random_state=42
    )
}

# Train models and make predictions
predictions = {}
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_scaled, y_train)
    predictions[name] = model.predict(X_test_scaled)
    
    # Calculate and print R² score for training data
    train_pred = model.predict(X_train_scaled)
    r2 = r2_score(y_train, train_pred)
    print(f"{name} R² score: {r2:.4f}")

# Create ensemble prediction (weighted average)
weights = {
    'xgb': 0.5,    # XGBoost usually performs best
    'gbm': 0.3,    # GBM second
    'rf': 0.2      # RF third
}

final_predictions = np.zeros(len(X_test))
for name, pred in predictions.items():
    final_predictions += weights[name] * pred

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_df['id'],
    'flood_probability': final_predictions
})

# Ensure predictions are within valid range [0, 1]
submission['flood_probability'] = submission['flood_probability'].clip(0, 1)

# Save predictions
submission.to_csv('ezpz.csv', index=False)
print("Submission file created successfully!")


Training data columns: ['id', 'precipitation_level', 'drainage_index', 'status_of_hydro_project', 'forest_degradation_levels', 'urban_density', 'climate_impact_score', 'infra_intensity', 'rate_of_sedimentation', 'farming_index', 'land_use_conflicts', 'emergency_readiness_index', 'stormwater_capacity', 'costal_risk_index', 'geoharazd_index', 'catchment_efficiency', 'infra_index', 'population_impact', 'ecosystem_depletion', 'deficiency_of_development', 'effectiveness_of_governance', 'flood_probability']


C:\Users\User\AppData\Local\Temp\ipykernel_18156\3525002057.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col1}_{col2}_interact'] = df[col1] * df[col2]
C:\Users\User\AppData\Local\Temp\ipykernel_18156\3525002057.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col1}_{col2}_ratio'] = df[col1] / (df[col2] + 1e-8)
C:\Users\User\AppData\Local\Temp\ipykernel_18156\3525002057.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

Training xgb...
xgb R² score: 0.8519
Training gbm...
